In [2]:
import json
import re
import numpy as np 
import pandas as pd

In [3]:
import importlib.util as imp

# The name 'twitter-connection.connection' seems inconsequential
#   perhaps just a spec name?
spec = imp.spec_from_file_location(
    'twitter-connection.connection', 
    '/home/rimov/Documents/Code/NLP/lin-que-dropping/twitter-connection/connection.py')
twit = imp.module_from_spec(spec)
spec.loader.exec_module(twit)

### References

https://twitter-trends.iamrohit.in/

In [4]:
cred_path = r'../twitter-connection/credentials.txt'

In [5]:
with open('../extraction/verb-stem-clean.txt') as f:
    verb_stem = json.load(f)

### Test

In [30]:
query_cond = ' (lang:es OR lang:pt) has:geo -is:retweet -has:links '
# fields_tweet = 'tweet.fields=lang,text,public_metrics,organic_metrics,geo,referenced_tweets'
fields_tweet = 'tweet.fields=lang,geo,public_metrics,referenced_tweets'
fields_expan = 'expansions=author_id,geo.place_id,entities.mentions.username'
fields_user = 'user.fields=created_at,location,public_metrics'
fields_place = 'place.fields=country'

In [31]:
conn_acad = twit.TwitterConnection(
    "spanish", 
    is_archive=True, 
    cred_path=cred_path,
    cred_prefix='PROF')

In [32]:
conn_acad.set_query(conditions=query_cond)
conn_acad.set_fields(tweet=fields_tweet, 
                      expansions=fields_expan, 
                      user=fields_user,
                      place=fields_place)

In [33]:
s = None
for vs in verb_stem.items():
    s = vs[1]
    break

s = '(' + s + ')'
s

'(ve OR vi OR vê)'

In [34]:
conn_acad.connect(s, is_next=True)
conn_acad.url

'https://api.twitter.com/2/tweets/search/all?query=(ve OR vi OR vê)&tweet.fields=lang,geo,public_metrics,referenced_tweets&expansions=author_id,geo.place_id,entities.mentions.username&user.fields=created_at,location,public_metrics&place.fields=country'

In [44]:
desc = pd.json_normalize(conn_acad.response)
data = pd.json_normalize(
    conn_acad.response, record_path='data')
place = pd.json_normalize(
    conn_acad.response, meta=['includes', 'places'], meta_prefix=True)

In [45]:
display(desc)
display(data.head(3))
display(place.head(3))

,data,includes.users,includes.places,meta.newest_id,meta.oldest_id,meta.result_count,meta.next_token
0,"[{'entities': {'mentions': [{'start': 3, 'end'...","[{'public_metrics': {'followers_count': 149, '...","[{'country': 'United Kingdom', 'full_name': 'N...",1407363174479937544,1407363173854949379,10,b26v89c19zqg8o3fpdg9v3fy1ozifi8qnft562xrhhsvx


,text,lang,id,author_id,referenced_tweets,entities.mentions,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,geo.place_id
0,"RT @RelusionH: Sakura, Juri and R.Mika, some o...",en,1407363174479937544,1020002616087064578,"[{'type': 'retweeted', 'id': '1407060521686294...","[{'start': 3, 'end': 13, 'username': 'Relusion...",108,0,0,0,NaN
1,RT @WasOnce_Skinny: Ajax Cape Town laid there ...,en,1407363174479896580,3416123883,"[{'type': 'retweeted', 'id': '1407351756661661...","[{'start': 3, 'end': 18, 'username': 'WasOnce_...",11,0,0,0,NaN
2,@Deputy_new It can’t only be Tuesday? I’ve wo...,en,1407363174454677506,20125015,"[{'type': 'replied_to', 'id': '140736176331465...","[{'start': 0, 'end': 11, 'username': 'Deputy_n...",0,0,0,0,67bc7263f7b9047b


,data,includes.users,includes.places,meta.newest_id,meta.oldest_id,meta.result_count,meta.next_token
0,"[{'entities': {'mentions': [{'start': 3, 'end'...","[{'public_metrics': {'followers_count': 149, '...","[{'country': 'United Kingdom', 'full_name': 'N...",1407363174479937544,1407363173854949379,10,b26v89c19zqg8o3fpdg9v3fy1ozifi8qnft562xrhhsvx


### Extraction

In [54]:
connection = twit.TwitterConnection(
    "spanish", 
    is_archive=True, 
    cred_path=cred_path,
    cred_prefix='PROF')

In [55]:
query_cond = ' (lang:es OR lang:pt) has:geo -is:retweet -has:links '
# fields_tweet = 'tweet.fields=lang,text,public_metrics,organic_metrics,geo,referenced_tweets'
fields_tweet = 'tweet.fields=lang,geo,public_metrics,referenced_tweets'
fields_expan = 'expansions=author_id,geo.place_id,entities.mentions.username'
fields_user = 'user.fields=created_at,location,public_metrics'
fields_place = 'place.fields=country'

In [56]:
connection.set_query(conditions=query_cond)
connection.set_fields(tweet=fields_tweet, 
                      expansions=fields_expan, 
                      user=fields_user,
                      place=fields_place)

In [51]:
# Dataframes for initial parsing
response = pd.DataFrame()
desc = pd.DataFrame()

In [57]:
idx = 0

for vs in verb_stem.items():
    if idx==1:
        break
    
    verb = vs[0]
    stems = '(' + vs[1] + ')'
    print(f'Retrieving tweets: {verb}\nTopics: {stems}')
    
    data = pd.DataFrame()
    
    while connection.connect(stems, is_next=True):
        desc = pd.json_normalize(
                connection.response)
        
        data = data.append(
            pd.json_normalize(
                connection.response, record_path='data'))
        
        data.reset_index(drop=True, inplace=True)
        
        # Duplicated text
#         dup = d.loc[:, 'text'].duplicated() 
        # Wrong language
        lang = (data.loc[:, 'lang']!='es') & (data.loc[:, 'lang']!='pt')
        # Is a retweet
        retweet = data.loc[:, 'text'].str.contains('RT @')
        
        # Remove '@...' mentions
        data.loc[:, 'text'] = data.loc[:, 'text'].str.replace(r'(@[\w]+ )', '', regex=True)
        
        print('\nUndesirables:')
        display(
            pd.concat([data.loc[:, ['lang', 'text']], 
                       lang.rename('is_wrong_lang'), 
                       retweet.rename('is_retweet')], 
                      axis=1))
        
        data.drop(
            data.loc[lang|retweet].index, 
            inplace=True)
        
        print(f'\nAfter dropping: {data.shape[0]}')
#         display(data.head())
        
        if data.shape[0] >= 5:
            break
    
    idx+=1
    response = response.append(data, ignore_index=True)

Retrieving tweets: ver
Topics: (ve OR vi OR vê)

Undesirables:


,lang,text,is_wrong_lang,is_retweet
0,pt,"RT @gigifxp: Ridicula, brigando por causa de m...",False,True
1,en,RT @brettdrc: I know someone that was employed...,True,True
2,en,"Yes bro, name 1 good team you’ve been in",True,False
3,tr,RT @opdrokanozdemir: Turistler Türkiye’ye gire...,True,True
4,tr,RT @Ahaber: Devlet Bahçeli'den Deniz Poyraz'ın...,True,True
5,tr,"$btc bakın Arkadaşlar...Lanet, ismi bile duyul...",True,False
6,es,#Deportes - Messi alcanza a Mascherano en réco...,False,False
7,es,Se ve poca gente en la calle !!!,False,False
8,tr,RT @banu3960: Daha fazlasi icin Rt fav ve yoru...,True,True
9,tr,"RT @MHP_Bilgi: Eğer hukuk varsa, eğer adalet h...",True,True



After dropping: 2

Undesirables:


,lang,text,is_wrong_lang,is_retweet
0,es,#Deportes - Messi alcanza a Mascherano en réco...,False,False
1,es,Se ve poca gente en la calle !!!,False,False
2,tr,RT @myeneroglu: Parti binasında vahşice katled...,True,True
3,en,With a power of your community you've reached ...,True,False
4,tr,RT @cazeeri: Poyraz eğer ayşeye engel olup ale...,True,True
5,en,Like + RT if you've never seen my tits! I'll s...,True,False
6,pt,vi ele hj. to feliz.,False,False
7,en,RT @jamesramertps: Project Brisa resulted in t...,True,True
8,tr,RT @kacsaatolduson: Kayyum rektör Melih Bulu'y...,True,True
9,en,RT @BobStoopsStatue: You could’ve at least men...,True,True



After dropping: 3
Error establishing connection:

 429
{"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"}


AttributeError: 'NoneType' object has no attribute 'json'

In [58]:
print(connection.url)

https://api.twitter.com/2/tweets/search/all?query=(ve OR vi OR vê)&next_token=b26v89c19zqg8o3fpdg9v3fy4rxkrq7j7hxuyo6433cl9&tweet.fields=lang,geo,public_metrics,referenced_tweets&expansions=author_id,geo.place_id,entities.mentions.username&user.fields=created_at,location,public_metrics&place.fields=country


In [59]:
response.head()

""


In [59]:
lang = response.loc[:, 'lang']!='es'

In [60]:
response.drop(response.loc[lang, :].index, inplace=True)
response

,lang,id,conversation_id,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,in_reply_to_user_id
4,es,1404865872515194884,1404865188109684747,"se ve tan real, además L y H se ven muy guapos",0,1,1,0,920381329669226497
32,es,1404865864143343620,1404631106876542976,Fraude es tu vida se sabia que mucha gente en ...,0,1,0,0,1252820905396301835
41,es,1404865862096535563,1404564648083525634,Sí ve a decirle a tu lider que le has defendid...,0,1,0,0,1362009157558292483
44,es,1404865862012702722,1404719991237251078,Y si su comportamiento es igual al este papelo...,0,0,1,0,1281968549330857986


In [61]:
response.loc[:, 'text'] = response.loc[:, 'text'].str.replace(r'(@\w+)\b', '', regex=True)
response

,lang,id,conversation_id,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,in_reply_to_user_id
4,es,1404865872515194884,1404865188109684747,"se ve tan real, además L y H se ven muy guapos",0,1,1,0,920381329669226497
32,es,1404865864143343620,1404631106876542976,Fraude es tu vida se sabia que mucha gente en ...,0,1,0,0,1252820905396301835
41,es,1404865862096535563,1404564648083525634,Sí ve a decirle a tu lider que le has defendid...,0,1,0,0,1362009157558292483
44,es,1404865862012702722,1404719991237251078,Y si su comportamiento es igual al este papelo...,0,0,1,0,1281968549330857986


In [63]:
response.loc[:, 'text'].apply(split)

NameError: name 'split' is not defined

In [62]:
for a in response.loc[:, 'text']:
    print(f'{a}\n')

se ve tan real, además L y H se ven muy guapos

Fraude es tu vida se sabia que mucha gente en el sur está votando por castillo qué pasa se ve que no has viajado por la ciudades

Sí ve a decirle a tu lider que le has defendido bien.

Y si su comportamiento es igual al este papelon. Obviamente que si. O pensas que se critica lo que se ve con hormonas ?? 🤔



In [11]:
with open('es_data.txt', 'w') as d:
    d.writelines(response.to_json(orient='table', force_ascii=False))

### Extracting Conversations

In [13]:
conv_ids = response.loc[:, 'conversation_id'].values

print(f'Extracted {len(conv_ids)} convos\n')

Extracted 22 convos



In [14]:
query_conv = 'conversation_id:'
query_conv_cond = ' -has:links '
conv_fields = 'tweet.fields=conversation_id,lang,in_reply_to_user_id,text,attachments,public_metrics'\
  '&expansions=author_id&user.fields=username,public_metrics'

In [19]:
# Dataframe for conversations
convs = pd.DataFrame()
# Dataframes for users
users = pd.DataFrame()

In [20]:
# If the rate limit is hit while querying, save progress
convo_idx = 0

In [21]:
for c_id in conv_ids[convo_idx:]:
    d = pd.DataFrame()
    u = pd.DataFrame()
    
    print(c)
    
    while connection.connect(query_conv + c_id + query_conv_cond, conv_fields, True):
        d = d.append(
            pd.json_normalize(connection.response, record_path='data'))
        u = u.append(
            pd.json_normalize(connection.response['includes'], record_path='users'))
        
        d.loc[:, 'text'] = d.loc[:, 'text'].str.replace(r'^(@[\w]+ )+', '', regex=True)
        
        short_text = d.loc[:, 'text'].apply(len) < 20
        lang = d.loc[:, 'lang']!='es'
        
        d.drop(
            d.loc[lang|short_text].index, 
            inplace=True)
        
        if d.shape[0]>14:
            break
        
    convs = convs.append(d, ignore_index=True)
    users = users.append(u, ignore_index=True)
    
    convo_idx+=1 

1402642305522282500
No next token!
1402637774805667841
No next token!
1402634503173165058
No next token!
1402624093946527745
1402634186909962240
No next token!
1402634112557518854
No next token!
1402633756888944642
No next token!
1402631188284645382
1402633207640641538
No next token!
1402626496989179915
1402629532994125827
No next token!
1402615477172686850
No next token!
1402625048180047879
No next token!
1402624924129366018
No next token!
1402617093514727426
1402131777641193477
No next token!
1402620991369535495
No next token!
1402619563200729092
No next token!
1402623588176478209
No next token!
1402623338535604229
No next token!
1402623254767050756
No next token!
1402454421439717378
No next token!


In [22]:
print(f'Extracted {convs.shape[0]} texts\nSample:\n')
convs.head()

Extracted 109 texts
Sample:



,in_reply_to_user_id,conversation_id,author_id,lang,text,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count
0,69416519,1402624093946527745,1537353780,es,"Muchas fuerza Florencia,lo que te desea el mal...",1402658383950454791,0,0,0,0
1,69416519,1402624093946527745,468895557,es,El mal que hace la madre lo pagan los hijos.\n...,1402656218150285315,0,0,0,0
2,69416519,1402624093946527745,1471308854,es,Nunca un hospital público estos garcas. Ni ver...,1402655563792764935,0,0,0,0
3,69416519,1402624093946527745,398461357,es,Porque no muestra la cara ?,1402653558219194372,0,0,0,0
4,69416519,1402624093946527745,169167606,es,Esta mujer esta anoréxica,1402653194174468103,0,0,1,0


In [23]:
with open('es_convs.txt', 'w') as d:
    d.writelines(convs.to_json(orient='table', force_ascii=False))

#### Examining extracted users

## TODO: users were improperly counted; count is WRONG

In [25]:
dups = users.loc[:, 'id'].duplicated()

print(f'Users: {users.shape[0]}, duplicated: {dups.sum()}')

Users: 256, duplicated: 27


In [27]:
users.drop(users.loc[dups].index, inplace=True)
print(users.shape[0])

229


In [28]:
with open('es_users.txt', 'w') as d:
    d.writelines(users.to_json(orient='table', force_ascii=False))